In [1]:
from src.jdv_conn import query_JDV
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option("display.colheader_justify","right")

# Load kcs and cases from the database

In [2]:
import json
kcs_rule_dict = json.load(open( "../data/kcs_rule_dict.json"))
kcs_string = ', '.join(kcs_rule_dict.keys())
#select 10 for tests
kcs_string = '740323, 24651, 1614393, 2065483, 797553, 3415331, 3446331, 4455551, 3991451, 29894'
kcs_rule_dict = {"740323": "filesystem/mounting_fs_with_errors.py", "24651": "filesystem/ext3_lookup_logs_unlinked_inode_error.py", "1614393": "filesystem/xfs_blocksize_issue.py", "2065483": "filesystem/nfs_fragment_too_large_error.py", "797553": "filesystem/nt_status_account_locked_out.py", "3415331": "filesystem/smb2_ntlm_mount_issue.py", "3446331": "filesystem/system_crash_is_size_safe_to_change.py", "4455551": "filesystem/smb3_systemd_issue.py", "3991451": "filesystem/cifs_fips_issue.py", "29894": "filesystem/ext_dx_add_entry.py"}

In [3]:
sql ="""
        SELECT kcs.resource_display_id__c, c.casenumber, c.subject, c.description
        FROM (SELECT  id, casenumber, subject, description
              FROM stg_gss_case
              WHERE stg_curr_flg = true
                  AND case_language__c='en'
                  AND isdeleted = false
                  AND ownerid != '00GA0000000XxxNMAS') c
            INNER JOIN stg_gss_case_rsrc_rltnshp kcs ON kcs.case__c = c.id
        WHERE stg_curr_flg = true
            AND isdeleted = false
            AND type__c = 'Link'  
            AND kcs.resource_display_id__c IN (""" + kcs_string + """)
        ORDER BY kcs.resource_display_id__c
"""

In [4]:
cases = query_JDV(queryString=sql)

In [5]:
cases

,resource_display_id__c,casenumber,subject,description
0,1614393,02144975,mount XFS failed: Function not implemented wit...,What problem/issue/behavior are you having tro...
1,1614393,02168396,Disk partitioning - Failed if I use 16K blocksize,"Hi Team,\n\nWe have RHEL 7.5 VM and attached m..."
2,1614393,02532866,We set the block size doesn't reflect to the x...,What problem/issue/behavior are you having tro...
3,1614393,02410915,Want to know correct syntax to format a logica...,What problem/issue/behavior are you having tro...
4,1614393,01734684,Failed to mount XFS file system with 16K block...,What problem/issue/behavior are you having tro...
5,1614393,01906558,Unable to increase XFS block size to 64KB beca...,What problem/issue/behavior are you having tro...
6,1614393,02073948,"mkfs.xfs -f -b size=32k produces ""Function not...",What problem/issue/behavior are you having tro...
7,1614393,02122142,Unable to mount xfs file system if the volume...,What problem/issue/behavior are you having tro...
8,1614393,02468840,Is possible reduce the bs in xfs?,¿Qué problema/comportamiento le está causando ...
9,1614393,02333745,Partition Table Issue for UEFI and 16 Bloxk Size,What problem/issue/behavior are you having tro...


# Single Rank Functions

In [6]:
# skips useless warnings in the pke methods
import logging
logging.basicConfig(level=logging.CRITICAL)
import pke

/root/sbr_top_issue/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
def preprocessing(text):
    #Replace newlines
    text = text.replace('\n', ' ')
    
    #Remove these unnecessary info from case description that is only need by the support associates
    #For resolution, the issue that customer has is what we need
    # We are going to use this additional text info as seperators and replace with some pattern
    seps = ['What problem/issue/behavior are you having trouble with?  What do you expect to see?', 'Where are you experiencing the behavior?  What environment?', 'When does the behavior occur? Frequently?  Repeatedly?   At certain times?', 'What information can you provide around timeframes and the business impact?']
    for sep in seps:
        text = text.replace(sep, '|#|')
        text_list = [each.strip() for each in text.split('|#|') if each.strip()]
        #print(text_list)
        #print(f"Actual Issue =>>> {text_list[0]}")
        issue = text_list[0]
    return issue

In [8]:
#Execute single rank algorithm to extract key phrases
def keyphrases(text):
    
    # define the set of valid Part Of Speech tags 
    pos = {'NOUN', 'PROPN', 'ADJ'}
    
    #create a SingleRank extractor
    singleRank_extractor = pke.unsupervised.SingleRank()
    
    # load the content of the document
    singleRank_extractor.load_document(input=text, language='en', normalization=None)
    
    # candidate selection (select the longest sequences of nouns and adjectives as candidates)
    singleRank_extractor.candidate_selection(pos)
    
    # candidate_weighing
    # candidate phrases are weighted using sum of their word's scores computed
    # using random walk. In graph, nodes are words of certain part-of-speech(nouns & adjectives)
    # that are connected if they occur in a window of 10 words
    singleRank_extractor.candidate_weighting(window=10, pos=pos)
    
    # rank the keyphrase and get the 10-higest scored candidates
    keyphrases_with_scores = singleRank_extractor.get_n_best(n=10)
    phrases = [keyphrase for keyphrase, score in keyphrases_with_scores]
    
    return phrases

In [9]:

def add_phrases(df):
    df['subject_key_phrases'] = df['subject'].apply(lambda x: keyphrases(preprocessing(x)))
    df['description_key_phrases'] = df['description'].apply(lambda x: keyphrases(preprocessing(x)))
    return df

# Calculate Keywords for cases

In [10]:
import time
start_time = time.time()
df = add_phrases(cases)
elapsed_time = time.time() - start_time

In [11]:
df['description_key_phrases'] = df.apply(lambda x: x['subject_key_phrases'] + x['description_key_phrases'],  axis=1)

In [12]:
df.style.set_properties(**{'text-align': 'left'})


,resource_display_id__c,casenumber,subject,description,subject_key_phrases,description_key_phrases
0,1614393,02144975,mount XFS failed: Function not implemented with block size set to 16K,"What problem/issue/behavior are you having trouble with? What do you expect to see? ds-san-dev-lux-01$ sudo mkfs.xfs -b size=16k /dev/xfs_vg01/xfs_lv01 Enter password for yuhuang (QUALPASS): meta-data=/dev/xfs_vg01/xfs_lv01 isize=512 agcount=32, agsize=10485600 blks = sectsz=512 attr=2, projid32bit=1 = crc=1 finobt=0, sparse=0 data = bsize=16384 blocks=335539200, imaxpct=5 = sunit=1 swidth=1024 blks naming =version 2 bsize=16384 ascii-ci=0 ftype=1 log =internal log bsize=16384 blocks=130432, version=2 = sectsz=512 sunit=1 blks, lazy-count=1 realtime =none extsz=16384 blocks=0, rtextents=0 ds-san-dev-lux-01$ sudo mkdir /local/syncdata02 ds-san-dev-lux-01$ sudo mount /dev/xfs_vg01/xfs_lv01 /local/syncdata02 mount: mount /dev/mapper/xfs_vg01-xfs_lv01 on /local/syncdata02 failed: Function not implemented ds-san-dev-lux-01$ Where are you experiencing the behavior? What environment? try to mount XFS with 16k block size. [root@ds-san-dev-lux-01 ~]# cat /etc/release Red Hat Enterprise Linux Server release 7.5 (Maipo) [root@ds-san-dev-lux-01 ~]# When does the behavior occur? Frequently? Repeatedly? At certain times? try to mount XFS with 16k block size. [root@ds-san-dev-lux-01 ~]# cat /etc/release Red Hat Enterprise Linux Server release 7.5 (Maipo) [root@ds-san-dev-lux-01 ~]# What information can you provide around timeframes and the business impact? the problem is repeatable.","['block size', 'mount xfs', 'function', 'k']","['block size', 'mount xfs', 'function', 'k', 'sudo mkfs.xfs -b', 'sudo mount', 'sunit=1 blks', 'syncdata02 mount', 'rtextents=0 ds', 'ftype=1 log', 'internal log', 'sudo mkdir', 'agsize=10485600 blks', 'bsize=16384']"
1,1614393,02168396,Disk partitioning - Failed if I use 16K blocksize,"Hi Team, We have RHEL 7.5 VM and attached multiple disks. I wanted to partitioning with 64K blocksize for each file system. I am not able to set 64K block size for file system and it is throuwing 'mount failed: Function not implemented' while mounting 64K blksize LVM Please help me to resolve this and provide the steps. Please reach Santosh +61 44 91 91 895 for this issue and updated his mail id.","['k blocksize', 'partitioning']","['k blocksize', 'partitioning', 'k block size', 'k blksize lvm', 'file system', 'k blocksize', 'santosh +61', 'multiple disks', 'issue', 'able', 'vm', 'rhel']"
2,1614393,02532866,We set the block size doesn't reflect to the xfs system.,"What problem/issue/behavior are you having trouble with? What do you expect to see? We set the block size doesn't reflect to the xfs system. [root@cmtoldbcmgpsg01 ~]# /sbin/blockdev --getra /dev/mapper/vg_wi_ssd-lv_data01 16384 [root@cmtoldbcmgpsg01 ~]# xfs_info /dev/mapper/vg_wi_ssd-lv_data01 meta-data=/dev/mapper/vg_wi_ssd-lv_data01 isize=512 agcount=32, agsize=48017600 blks = sectsz=4096 attr=2, projid32bit=1 = crc=1 finobt=0 spinodes=0 data = bsize=4096 blocks=1536563200, imaxpct=5 = sunit=64 swidth=256 blks naming =version 2 bsize=4096 ascii-ci=0 ftype=1 log =internal bsize=4096 blocks=521728, version=2 = sectsz=4096 sunit=1 blks, lazy-count=1 realtime =none extsz=4096 blocks=0, rtextents=0 Where are you experiencing the behavior? What environment? System slow. When does the behavior occur? Frequently? Repeatedly? At certain times? N/A What information can you provide around timeframes and the business impact? Need to fix it for performance issue asap.","['xfs system', 'block size']","['xfs system', 'block size', 'ci=0 ftype=1 log', 'finobt=0 spinodes=0 data', 'bsize=4096', 'count=1 realtime', 'sunit=1 blks', 'mapper', 'xfs system', 'lv_data01', 'sectsz=4096', 'blockdev --getra']"
3,1614393,02410915,Want to know correct syntax to format a logical volume with xfs filesystem with specific block size,"What problem/issue/behavior are you having trouble with? What do you expect to see? C

In [13]:
elapsed_time

229.93262028694153

# Calculate Keywords for KCS/rule

In [14]:
df[df['resource_display_id__c'] == '1614393'].shape[0]

29

In [15]:
# 遍历df
rule_tag_dict = {}
for key in kcs_rule_dict.keys():
    temp_df = df[df['resource_display_id__c'] == key]
    case_count = temp_df.shape[0]
    train_count = int(case_count * (4/5))
    temp_train_df = temp_df[:train_count-1]
    kcs_tag = []
    for description_phrases in temp_train_df['description_key_phrases']:
        kcs_tag = kcs_tag + description_phrases    
    rule_tag_dict[key] = kcs_tag   
        

In [16]:
rule_tag_dict

{'740323': ['filesystem',
  'high utilization',
  'filesystem utilization',
  'df -g output',
  'actual size',
  'gb',
  'file system',
  'mail service',
  'regards',
  'zimbra',
  'prioroty',
  'server',
  'e2fsck',
  'checktime',
  'ext3-fs',
  'servers unable',
  'jump server',
  'below output',
  'servers unable',
  'gpd-973',
  'a9e',
  'ff03',
  'mode',
  'team',
  'issue',
  'high swap usage',
  'filesystem errors',
  'logs',
  'swap usage',
  'oracle databases',
  'failover cluster',
  'usage',
  'filesystem errors',
  'node active',
  'few days',
  'server logs',
  'rhel',
  'issue',
  'hardware error',
  'performance issue',
  'dmesg',
  'machine check events',
  'hardware error',
  'root@pruswipprodb1',
  'strange errors',
  'message.log',
  '18th sep log',
  'umount messages',
  'filesystem mount',
  'message.log',
  'reason',
  'possible currupt file system',
  'fsck',
  'mount point',
  'netbackup',
  'root file system',
  'issue',
  'maintenance mode',
  'sos report',
  

# Calculate the match percentage between 'new' cases and rule

In [18]:
performance = []
for key in kcs_rule_dict.keys():
    temp_df = df[df['resource_display_id__c'] == key]
    case_count = temp_df.shape[0]
    train_count = int(case_count * (4/5))
    temp_test_df = temp_df[train_count:]
    for description_phrases in temp_test_df['description_key_phrases']:
        performance.append({'KCS':key, 'case key phrases number': len(description_phrases), 'case matched tags number': len(list(set(description_phrases) & set(rule_tag_dict[key]))), 'rule tags number': len(rule_tag_dict[key])})
      
performance_df = pd.DataFrame(performance)
def calculate_hit_rate(row):
    if row['rule tags number'] == 0:
        return 0
    return row['case matched tags number']/row['rule tags number']
    
performance_df['hit_rate'] = performance_df.apply(calculate_hit_rate, axis=1)
performance_df

,KCS,case key phrases number,case matched tags number,rule tags number,hit_rate
0,740323,8,1,144,0.006944
1,740323,3,1,144,0.006944
2,740323,13,6,144,0.041667
3,740323,12,0,144,0.000000
4,740323,15,2,144,0.013889
5,24651,13,5,176,0.028409
6,24651,12,3,176,0.017045
7,24651,13,2,176,0.011364
8,24651,11,2,176,0.011364
9,24651,5,1,176,0.005682


# The average match percentage is close to 0

In [19]:
performance_df['hit_rate'].mean()

0.009440565418467538

In [20]:
performance_df['hit_rate'].max()

0.041666666666666664